In [3]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [5]:
# 마우스와 그랩컷으로 배경과 전경 분리 (grabcut.py)

img = cv2.imread('data/image_0001.jpg')
img_draw = img.copy()
mask = np.zeros(img.shape[:2], dtype=np.uint8)  # 마스크 생성
rect = [0,0,0,0]    # 사각형 영역 좌표 초기화
mode = cv2.GC_EVAL  # 그랩컷 초기 모드
# 배경 및 전경 모델 버퍼
bgdmodel = np.zeros((1,65),np.float64)
fgdmodel = np.zeros((1,65),np.float64)

# 마우스 이벤트 처리 함수
def onMouse(event, x, y, flags, param):
    global mouse_mode, rect, mask, mode
    if event == cv2.EVENT_LBUTTONDOWN : # 왼쪽 마우스 누름
        if flags <= 1: # 아무 키도 안 눌렀으면
            mode = cv2.GC_INIT_WITH_RECT # 드래그 시작, 사각형 모드 ---①
            rect[:2] = x, y # 시작 좌표 저장
    # 마우스가 움직이고 왼쪽 버튼이 눌러진 상태
    elif event == cv2.EVENT_MOUSEMOVE and flags & cv2.EVENT_FLAG_LBUTTON :
        if mode == cv2.GC_INIT_WITH_RECT: # 드래그 진행 중 ---②
            img_temp = img.copy()
            # 드래그 사각형 화면에 표시
            cv2.rectangle(img_temp, (rect[0], rect[1]), (x, y), (0,255,0), 2)
            cv2.imshow('img', img_temp)
        elif flags > 1: # 키가 눌러진 상태
            mode = cv2.GC_INIT_WITH_MASK    # 마스크 모드 ---③
            if flags & cv2.EVENT_FLAG_CTRLKEY :# 컨트롤 키, 분명한 전경
                # 흰색 점 화면에 표시
                cv2.circle(img_draw,(x,y),3, (255,255,255),-1)
                # 마스크에 GC_FGD로 채우기      ---④
                cv2.circle(mask,(x,y),3, cv2.GC_FGD,-1)
            if flags & cv2.EVENT_FLAG_SHIFTKEY : # 쉬프트키, 분명한 배경
                # 검정색 점 화면에 표시
                cv2.circle(img_draw,(x,y),3, (0,0,0),-1)
                # 마스크에 GC_BGD로 채우기      ---⑤
                cv2.circle(mask,(x,y),3, cv2.GC_BGD,-1)
            cv2.imshow('img', img_draw) # 그려진 모습 화면에 출력
    elif event == cv2.EVENT_LBUTTONUP: # 마우스 왼쪽 버튼 뗀 상태 ---⑥
        if mode == cv2.GC_INIT_WITH_RECT : # 사각형 그리기 종료
            rect[2:] =x, y # 사각형 마지막 좌표 수집
            # 사각형 그려서 화면에 출력 ---⑦
            cv2.rectangle(img_draw, (rect[0], rect[1]), (x, y), (255,0,0), 2)
            cv2.imshow('img', img_draw)
        # 그랩컷 적용 ---⑧
        cv2.grabCut(img, mask, tuple(rect), bgdmodel, fgdmodel, 1, mode)
        img2 = img.copy()
        # 마스크에 확실한 배경, 아마도 배경으로 표시된 영역을 0으로 채우기
        img2[(mask==cv2.GC_BGD) | (mask==cv2.GC_PR_BGD)] = 255
        cv2.imshow('grabcut', img2) # 최종 결과 출력
        cv2.imwrite('data/new.jpg', img2)
        mode = cv2.GC_EVAL # 그랩컷 모드 리셋

cv2.imshow('img', img)
cv2.setMouseCallback('img', onMouse)

while True:    
    if cv2.waitKey(0) & 0xFF == 27 : # esc
        break

cv2.destroyAllWindows()

In [7]:
win_title = 'resume photo'
alpha = 0.8

img2 = cv2.imread('data/new.jpg')
img3 = cv2.imread('data/m1.jpg')

# # resize
img2 = cv2.resize(img2, (300, 400))
img3 = cv2.resize(img3, (300, 400))

# 이미지 합성 (알파 블렌딩... 비트와이즈 이용할려고 계획중)
blended = cv2.addWeighted(img2, alpha, img3, (alpha), 0)
cv2.imshow('blended', blended)
cv2.waitKey(0)
cv2.destroyAllWindows()

# 미니 포토샵 만들기
## 1. 기능
- 모자이크 기능 (완료)
- 밝기 (완료)
- 흑백 기능 (완료)
- 영상 뒤집기 기능
- 리셋 기능 (완료)
- 확대/축소 (완료)
- 자동 저장 기능 (완료)
- 간단한 라인 그리기 
<!-- - 화질 개선 기능 (X) -->
<!-- - 스케치 기능 (X_시간 남으면) -->
<!-- - 리퀴파이 기능 (X) -->

In [17]:
import cv2
import numpy as np
import math
import matplotlib.pyplot as plt

name = input('사진을 고르시오.')
img = cv2.imread(f'data/{name}.jpg')
rows, cols = img.shape[:2]

# 밝기 조절
def on_bright(pos):
    global img # 밖에있는 img를 갖고 옴
    if pos == 0:
        img = cv2.imread(f'data/{name}.jpg')
        cv2.imshow('image', img)
    else:
        img = cv2.add(img, 10*pos)
        cv2.imshow('image', img)

# 흑백 처리
def gray_scale(pos):
    global img

    if pos == 1:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        cv2.imshow('image', img)
    
    elif pos == 0:
        img = cv2.imread(f'data/{name}.jpg')
        cv2.imshow('image', img)

# 모자이크 기능
def mosaic(pos):
    global img
    rate = 15 # 모자이크에 사용할 축소 비율 (1/rate)
    if pos == 1:
        x,y,w,h = cv2.selectROI('image', img, False) # 관심영역 선택
        if w and h:
            roi = img[y:y+h, x:x+w]   # 관심영역 지정
            roi = cv2.resize(roi, (w//rate, h//rate)) # 1/rate 비율로 축소
            # 원래 크기로 확대
            roi = cv2.resize(roi, (w,h), interpolation=cv2.INTER_AREA)  
            img[y:y+h, x:x+w] = roi   # 원본 이미지에 적용
            cv2.imshow('image', img)

# 확대/축소 (보완 필요)
def img_bigger(pos):
    global img
    if pos == 0:
        img = cv2.imread(f'data/{name}.jpg')
        cv2.imshow('image', img)
    else:
        pos += 1
        img = cv2.resize(img, None, None, pos, pos, cv2.INTER_CUBIC)
        cv2.imshow('image', img)

def img_smaller(pos):
    global img
    if pos == 0:
        img = cv2.imread(f'data/{name}.jpg')
        cv2.imshow('image', img)
    else:
        pos = pos*0.5
        img = cv2.resize(img, None, None, pos, pos, cv2.INTER_CUBIC)
        cv2.imshow('image', img)

# reset 기능
def reset(pos):
    global img
    if pos == 1:
        img = cv2.imread(f'data/{name}.jpg')
        cv2.imshow('image', img)

# save 기능
def save():
    global img
    img = img
    cv2.imwrite('data/new_img.jpg', img)

# drawing 기능
green = (0, 255, 0)
red = (0, 0, 255)
drawing = True
mode = True
ix, iy = 0, 0

# mouse callback function 
def draw_circle(event, x, y, flags, param):
    global img, ix, iy, drawing, mode
    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        ix, iy = x, y
    
    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        if mode == True:
            cv2.rectangle(img, (ix, iy), (x, y), red)
            cv2.imshow('image', img)
        else:
            radius = (ix-x)**2 + (iy-y)**2
            cv2.circle(img, (x, y), int(radius), green)
            cv2.imshow('image', img)


# 창이 생성된 이후에 호출
cv2.namedWindow('image')
cv2.resizeWindow(winname='image', width=300, height=300)
cv2.setMouseCallback('image', draw_circle)
cv2.createTrackbar('big', 'image', 0, 2, img_bigger)
cv2.createTrackbar('samll', 'image', 0, 2, img_smaller)
cv2.createTrackbar('bright', 'image', 0, 10, on_bright)
cv2.createTrackbar('gray', 'image', 0, 1, gray_scale)
cv2.createTrackbar('mosaic', 'image', 0, 1, mosaic)
cv2.createTrackbar('reset', 'image', 0, 1, reset)

# 결과
# cv2.waitKey(0)
# cv2.destroyAllWindows()
while True:
    k = cv2.waitKey(1) & 0xFF
    if k == ord('m'):
        mode = not mode
    elif k == 27:
        break
cv2.destroyAllWindows()

# 저장
if __name__ == '__main__':
    save()

In [ ]:
# 미완성 코드
# 그리기 기능
green = (0, 255, 0)
red = (0, 0, 255)
drawing = True
mode = True
(ix, iy) = (-1, -1)

# mouse callback function 
def draw_circle(event, x, y, flags, param):
    global img, ix, iy, drawing, mode
    if event == cv2.EVENT_LBUTTONDBLCLK:
        drawing = True
        ix, iy = x, y
        # cv2.circle(img, (x,y), 20, (255,0,0), 4)
    
    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        if mode == True:
            cv2.rectangle(img, (ix, iy), (x, y), red)
        
        else:
            radius = (ix-x)**2 + (iy-y)**2
            cv2.circle(img, (x, y), int(radius), green)


while True:
    k = cv2.waitKey(1) & 0xFF
    if k == ord('m'):
        mode = not mode
    elif k == 27:
        break

cv2.destroyAllWindows()

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

img = cv2.imread('data/image_0001.jpg')
rows, cols = img.shape[:2]

# Clahe 적용
img_yuv = cv2.cvtColor(blended, cv2.COLOR_BGR2YUV)

# 밝기 채널에 대해서 CLAHE 적용
img_clahe = img_yuv.copy()
clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8)) #CLAHE 생성
img_clahe[:,:,0] = clahe.apply(img_clahe[:,:,0])           #CLAHE 적용
img_clahe = cv2.cvtColor(img_clahe, cv2.COLOR_YUV2BGR)

# 결과 출력
cv2.imshow('CLAHE', img_clahe)
cv2.waitKey()
cv2.destroyAllWindows()

def onChange():
    pass

def trackBar():
    cv2.createTrackbar('Degree', 'Rotate', 0, 180, onChange)
    x = cv2.getTrackbarPos('Degree', 'Rotate')
    
    # 라디안 각도 계산(60진법을 호도법으로 변경)
    degree = cv2.getRotationMatrix2D((cols/2, rows/2), x, 1.0) 

    # 변환 행렬 적용
    rotate = cv2.warpAffine(img, degree, (cols, rows))
    

    cv2.imshow("rotate", rotate)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

trackBar()



error: OpenCV(4.5.3) C:\Users\runneradmin\AppData\Local\Temp\pip-req-build-1i5nllza\opencv\modules\highgui\src\window_w32.cpp:2359: error: (-27:Null pointer) NULL window: 'Rotate' in function 'icvCreateTrackbar'
